<a href="https://colab.research.google.com/github/sime1/notebooks/blob/master/Bayesian_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes - 20 newsgroups dataset 

In questo notebook applico i diversi metodi *Naive Bayes* forniti da `sklearn` al dataset [20 newsgroups](http://qwone.com/~jason/20Newsgroups/).

In [0]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

train_ds = fetch_20newsgroups(subset='train')
test_ds = fetch_20newsgroups(subset='test')

vectorizer.fit(train_ds.data)

X_train, y_train = vectorizer.transform(train_ds.data), train_ds.target
X_test, y_test = vectorizer.transform(test_ds.data), test_ds.target

Come è possibile vedere dal codice, ho deciso di utilizzare `TfidfVectorizer` per trasformare i testi dei messaggi nel dataset in vettori di feature, in modo da poterli utilizzare coi metodi di `sklearn`.

Ho utilizzato questo metodo invece che `CountVectorizer` (quello utilizzato dalla funzione `fetch_20newsgroups_vectorized`) perché ho notato che da risultati migliori.

## Multinomial NB

Questo metodo implementa l'algoritmo NB per dati che seguono una [distribuzione multinomiale](https://en.wikipedia.org/wiki/Multinomial_distribution). In particolare, è una delle varianti dell'algoritmo che viene applicata calssicamente a task di classificazione testuale.

In [2]:
from sklearn.naive_bayes import MultinomialNB

params = {
    # 1.0e-10 instead of 0 to avoid getting a warning
    'alpha': np.linspace(1.0e-10, 2.0, 20)
}

model = MultinomialNB()
sel = GridSearchCV(model, params, cv=5)
sel.fit(X_train, y_train)
mnb_model = sel.best_estimator_
mnb_model.score(X_test, y_test)

0.8255443441317047

# Complement NB

Questo metodo è un adattamento del *Multinomial NB* che funziona particolarmente bene con dataset non equilibrati

In [3]:
from sklearn.naive_bayes import ComplementNB

params = {
    # 1.0e-10 instead of 0 to avoid getting a warning
    'alpha': np.linspace(1.0e-10, 2.0, 20),
    'norm': [True, False]
}

model = ComplementNB()
sel = GridSearchCV(model, params, cv=5)
sel.fit(X_train, y_train)
cnb_model = sel.best_estimator_
cnb_model.score(X_test, y_test)

0.8315188528943176

## Bernoulli NB

Questo metodo implementa NB per dati che hanno una distribuzione di Bernoulli multivariata; questo significa che il metodo permette di avere più features, ma assume che ognuna di esse sia binaria 

In [4]:
from sklearn.naive_bayes import BernoulliNB

params = {
    'alpha': np.linspace(1.0e-10, 2.0, 10),
    'binarize': np.linspace(0.0, 1.0, 4)
}

model = BernoulliNB()
sel = GridSearchCV(model, params, cv=5)
sel.fit(X_train, y_train)
bnb_model = sel.best_estimator_
bnb_model.score(X_test, y_test)

0.7792087095061073

## Confronto risultati

In [5]:
print('Multinomial')
print(f'\tscore: {mnb_model.score(X_test, y_test)}')
print('\tparameters:')
print(f'\t\talpha:{mnb_model.alpha}')

print('Complement\n')
print(f'\tscore: {cnb_model.score(X_test, y_test)}')
print('\tparameters:')
print(f'\t\talpha:{cnb_model.alpha}')
print(f'\t\tnorm:{cnb_model.norm}')

print('Bernoulli\n')
print(f'\tscore: {bnb_model.score(X_test, y_test)}')
print('\tparameters:')
print(f'\t\talpha:{bnb_model.alpha}')
print(f'\t\tbinarize:{bnb_model.binarize}')



Multinomial
	score: 0.8255443441317047
	parameters:
		alpha:0.10526315798947368
Complement

	score: 0.8315188528943176
	parameters:
		alpha:0.10526315798947368
		norm:False
Bernoulli

	score: 0.7792087095061073
	parameters:
		alpha:1e-10
		binarize:0.0


### Note

* Non ho utilizzato nè *Gaussian NB* nè *Categorical NB* in quanto la loro implementazione in `sklearn` richiede dati in formato denso, e provare a trasformare i dati da matrice sparsa a densa richiede più RAM di quanta resa disponibile da Colab.